# Creating a Placement Algorithm

This tutorial demonstrates how we can create a simple placement algorithm on EdgeSimPy.

Let's start by importing the EdgeSimPy modules:

In [4]:
# EdgeSimPy Import Debugging Script

# Explicit dependency installation
!pip install rich
!pip install networkx==2.6.2
!pip install matplotlib pandas numpy
!pip install git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0

# Python and package information
!python --version
!pip list | grep -E "networkx|edge_sim_py"

# Comprehensive import and debugging script
import sys
import os
import importlib

def print_module_structure(module_name):
    """
    Recursively print the structure of a module
    """
    print(f"\n--- Module Structure for {module_name} ---")
    try:
        # Import the module
        module = importlib.import_module(module_name)

        # Get the module's file path
        module_file = getattr(module, '__file__', 'No __file__ attribute')
        print(f"Module file path: {module_file}")

        # Get the module's directory
        module_dir = os.path.dirname(module_file) if hasattr(module, '__file__') else 'Unknown'
        print(f"Module directory: {module_dir}")

        # List all attributes and their types
        print("\nModule Contents:")
        for attr_name in dir(module):
            try:
                attr = getattr(module, attr_name)
                print(f"  {attr_name}: {type(attr)}")
            except Exception as attr_err:
                print(f"  {attr_name}: Could not retrieve (Error: {attr_err})")

        # List files in the module directory
        if os.path.isdir(module_dir):
            print("\nFiles in module directory:")
            try:
                for item in os.listdir(module_dir):
                    print(f"  {item}")
            except Exception as list_err:
                print(f"  Could not list directory contents: {list_err}")

    except ImportError as e:
        print(f"Could not import {module_name}: {e}")
    except Exception as e:
        print(f"Unexpected error examining {module_name}: {e}")

# Print Python path and sys.path for debugging
print("--- Python Path ---")
print(sys.path)

# Attempt to import and examine EdgeSimPy
print_module_structure('edge_sim_py')

# Attempt alternative import methods
print("\n--- Alternative Import Attempts ---")
import_attempts = [
    'edge_sim_py',
    'edge_sim_py.core',
    'edge_sim_py.components',
    'edge_sim_py.device',
    'edge_sim_py.server'
]

for attempt in import_attempts:
    print(f"\nTrying to import {attempt}")
    try:
        module = importlib.import_module(attempt)
        print(f"Successfully imported {attempt}")
        print(f"Module file: {getattr(module, '__file__', 'No file attribute')}")
    except ImportError as e:
        print(f"Import failed: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# List installed packages with their paths
print("\n--- Installed Packages Paths ---")
for package_name in ['edge_sim_py', 'networkx', 'numpy', 'pandas']:
    try:
        package = importlib.import_module(package_name)
        print(f"{package_name}: {package.__file__}")
    except ImportError:
        print(f"{package_name}: Not found")
    except Exception as e:
        print(f"{package_name}: Error - {e}")


  Cloning https://github.com/EdgeSimPy/EdgeSimPy.git (to revision v1.1.0) to /tmp/pip-req-build-k7brjlxd
  Running command git clone --filter=blob:none --quiet https://github.com/EdgeSimPy/EdgeSimPy.git /tmp/pip-req-build-k7brjlxd
  Running command git checkout -q 5ea400b39390490b25dabf8be711fe559cb2cbff
  Resolved https://github.com/EdgeSimPy/EdgeSimPy.git to commit 5ea400b39390490b25dabf8be711fe559cb2cbff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Python 3.11.11
edge_sim_py                        1.1.0
networkx                           2.6.2
--- Python Path ---
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']

-

## Implementing the Placement Algorithm

In this example, we are going to create a simple placement algorithm that works according to the well-known First-Fit heuristic. In a nutshell, our algorithm will provision each service to the first edge server with available resources to host them.

In [5]:
def my_algorithm(parameters):
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break

## Running the Simulation

As we're creating a placement algorithm, we must instruct EdgeSimPy that it needs to continue the simulation until all services are provisioned within the infrastructure.

To do so, let's create a simple function that will be used as the simulation's stopping criterion. EdgeSimPy will run that function at the end of each time step, halting the simulation as soon as it returns `True`.

In [6]:
def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0

    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1

    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

Google Colab Setup for FCFS Task Processing



In [18]:
# Google Colab Setup for FCFS Task Processing

# Install required libraries
!pip install numpy

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# List files in the task sets directory
import os
task_sets_dir = '/content/drive/My Drive/FCFS_Task_Sets/'
print("Available task set files:")
for filename in os.listdir(task_sets_dir):
    print(filename)

# Note: After running this, copy the full path of the desired JSON file
# and use it in the main FCFS scheduler script

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Available task set files:
fcfs_task_set_20250201_201915.json


FCFS Algorithm Logic


In [29]:
import json
from typing import List, Dict, Any
import logging
import sys
import time

# Configure logging to print to console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)  # Explicitly add console output
    ]
)
logger = logging.getLogger(__name__)

# Print function to ensure output
def print_to_console(*args, **kwargs):
    """
    Wrapper function to ensure printing
    """
    print(*args, **kwargs)
    sys.stdout.flush()

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')
        self.task_class = self.details.get('task_class', 'generic')
        self.cpu_intensity = self.details.get('cpu_intensity', 'medium')

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with enhanced tracking and visualization
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.memory = memory
        self.bandwidth = bandwidth

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)

    def process_queue(self, current_time: float):
        """
        Process tasks in the queue with detailed tracking
        """
        # Process current tasks first
        for task in self.current_tasks[:]:
            processing_result = task.process(self.cpu_rating)

            # Detailed task processing output
            self._log_task_processing(task, processing_result)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

        # If resource has available capacity, move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:  # Limit concurrent tasks
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)

        return len(self.current_tasks)

    def _log_task_processing(self, task: Task, processing_result: Dict):
        """
        Log detailed task processing information
        """
        print_to_console(
            f"Resource {self.id} ({self.type}) - "
            f"Task {task.id} ({task.task_name}): "
            f"Processed {processing_result['processed']:.2f} MI, "
            f"Remaining {processing_result['remaining']:.2f} MI, "
            f"Completion: {processing_result['completion_percentage']:.2f}%"
        )

class AdvancedFCFSScheduler:
    """
    Advanced First-Come-First-Serve Scheduler with Real-Time Visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.task_queue: List[Task] = []
        self.current_time = 0

        # Metrics tracking with enhanced details
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'queued_tasks': 0,
            'task_distribution': {},
            'resource_utilization': {},
            'average_wait_time': 0,
            'max_wait_time': 0
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks from JSON with comprehensive parsing
        """
        print_to_console(f"Attempting to load tasks from: {json_path}")

        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),  # Default 10 MB
                cpu_required=task_dict.get('instructions', 50000),  # Default 50,000 MI
                task_details=task_dict
            )
            task.arrival_time = self.current_time
            tasks.append(task)

        print_to_console(f"Loaded {len(tasks)} tasks from JSON")
        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources with advanced visualization
        """
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin task distribution with visualization
        resource_index = 0
        for task in tasks:
            # Select resource
            resource = self.resources[resource_index]

            # Enqueue task
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1

            # Cycle through resources
            resource_index = (resource_index + 1) % len(self.resources)

        # Update metrics
        self.metrics['task_distribution'] = task_distribution
        self.metrics['queued_tasks'] = sum(len(resource.task_queue) for resource in self.resources)

        # Print initial distribution
        print_to_console("\n--- Initial Task Distribution ---")
        for resource_type, count in task_distribution.items():
            print_to_console(f"{resource_type}: {count} tasks")

        print_to_console("\n--- Resource Queue Lengths ---")
        for i, resource in enumerate(self.resources, 1):
            print_to_console(f"Resource {i} ({resource.type}) Queue Length: {len(resource.task_queue)} tasks")

    def run_simulation(self, max_iterations: int = 1000):
        """
        Run scheduling simulation with real-time visualization
        """
        # Distribute tasks initially
        self.distribute_tasks()

        # Simulation loop with enhanced visualization
        start_time = time.time()
        for iteration in range(max_iterations):
            print_to_console(f"\n--- Iteration {iteration} ---")

            # Process queues for all resources
            completed_in_iteration = 0
            resource_utilization = {}

            for resource in self.resources:
                # Track resource utilization
                initial_completed = len(resource.completed_tasks)
                resource.process_queue(self.current_time)
                completed_this_resource = len(resource.completed_tasks) - initial_completed
                completed_in_iteration += completed_this_resource

                # Calculate resource utilization
                resource_utilization[resource.type] = {
                    'completed_tasks': completed_this_resource,
                    'current_tasks': len(resource.current_tasks),
                    'queue_length': len(resource.task_queue)
                }

            # Update metrics
            self.metrics['completed_tasks'] = sum(
                len(resource.completed_tasks) for resource in self.resources
            )
            self.metrics['resource_utilization'] = resource_utilization

            # Print real-time resource utilization
            print_to_console("\n--- Resource Utilization ---")
            for resource_type, stats in resource_utilization.items():
                print_to_console(
                    f"{resource_type}: "
                    f"Completed: {stats['completed_tasks']}, "
                    f"Current Tasks: {stats['current_tasks']}, "
                    f"Queue Length: {stats['queue_length']}"
                )

            # Check if all tasks are processed
            if self.metrics['completed_tasks'] == self.metrics['total_tasks']:
                print_to_console("\n--- All Tasks Processed! ---")
                break

            # Increment time
            self.current_time += 1

            # Optional: Add a small delay to simulate real-time processing
            time.sleep(0.1)

        # Calculate total processing time
        total_processing_time = time.time() - start_time
        self.metrics['total_processing_time'] = total_processing_time

        # Calculate wait time metrics
        self.calculate_wait_time_metrics()

        return self.metrics

    def calculate_wait_time_metrics(self):
        """
        Calculate comprehensive wait time metrics
        """
        all_completed_tasks = []
        for resource in self.resources:
            all_completed_tasks.extend(resource.completed_tasks)

        if all_completed_tasks:
            wait_times = [task.wait_time for task in all_completed_tasks]
            self.metrics['average_wait_time'] = sum(wait_times) / len(wait_times)
            self.metrics['max_wait_time'] = max(wait_times)

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Explicitly set print to console
    print = print_to_console

    # Create resources matching original configuration
    resources = create_original_resources()

    # Print initial resource details
    print("\n--- Resource Configurations ---")
    for resource in resources:
        print(f"Resource {resource.id} ({resource.type}):")
        print(f"  CPU Rating: {resource.cpu_rating} MI/s")
        print(f"  Memory: {resource.memory} GB")
        print(f"  Bandwidth: {resource.bandwidth} MB/s")

    # Initialize scheduler
    scheduler = AdvancedFCFSScheduler(resources)

    # Run simulation
    metrics = scheduler.run_simulation()

    # Print final detailed metrics
    print("\n--- Final Scheduling Metrics ---")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    # Detailed resource reporting
    print("\n--- Final Resource Status ---")
    for resource in scheduler.resources:
        print(f"\nResource {resource.id} ({resource.type}):")
        print(f"Completed Tasks: {len(resource.completed_tasks)}")
        print(f"Remaining Queue: {len(resource.task_queue)}")

if __name__ == "__main__":
    main()


Strømmer utdata som er avkortet til de siste 5000 linjene.
Resource 1 (Edge_Raspberry_Pi) - Task 330 (Task_330): Processed 80000.00 MI, Remaining 600000.00 MI, Completion: 40.00%
Resource 1 (Edge_Raspberry_Pi) - Task 1326 (Task_1326): Processed 80000.00 MI, Remaining 100000.00 MI, Completion: 80.00%
Resource 1 (Edge_Raspberry_Pi) - Task 1023 (Task_1023): Processed 80000.00 MI, Remaining 920000.00 MI, Completion: 8.00%

--- Resource Utilization ---
Edge_Raspberry_Pi: Completed: 0, Current Tasks: 5, Queue Length: 360
Edge_Smartphone: Completed: 0, Current Tasks: 0, Queue Length: 0
Cloud_Host: Completed: 0, Current Tasks: 0, Queue Length: 0

--- Iteration 586 ---
Resource 1 (Edge_Raspberry_Pi) - Task 264 (Task_264): Processed 80000.00 MI, Remaining 80000.00 MI, Completion: 96.00%
Resource 1 (Edge_Raspberry_Pi) - Task 977 (Task_977): Processed 80000.00 MI, Remaining 2560000.00 MI, Completion: 36.00%
Resource 1 (Edge_Raspberry_Pi) - Task 330 (Task_330): Processed 80000.00 MI, Remaining 5200

Task Generator mixed large and small files


In [11]:
import random
import json
from typing import List, Dict, Any
import os
from collections import defaultdict

class TaskGenerator:
    def __init__(self):
        # Detailed task configurations with more specific characteristics
        self.task_configs = {
            # Large Read Tasks
            'large_read_tasks': {
                'RT1': {
                    'instr': 2_000_000,  # Million Instructions
                    'data': 5,           # GB
                    'cpu_intensity': 'high',
                    'memory_intensity': 'high',
                    'task_class': 'CPU-intensive, memory-intensive'
                },
                'RT2': {
                    'instr': 4_000_000,
                    'data': 0.2,
                    'cpu_intensity': 'high',
                    'memory_intensity': 'low',
                    'task_class': 'CPU-intensive, memory-light'
                }
            },
            # Large Write Tasks
            'large_write_tasks': {
                'WT1': {
                    'instr': 2_000_000,
                    'data': 2,
                    'cpu_intensity': 'high',
                    'io_intensity': 'high',
                    'task_class': 'CPU-intensive, I/O-intensive'
                },
                'WT2': {
                    'instr': 1_000_000,
                    'data': 0.5,
                    'cpu_intensity': 'high',
                    'io_intensity': 'low',
                    'task_class': 'CPU-intensive, I/O-light'
                }
            },
            # Small Read Tasks
            'small_read_tasks': {
                'RT3': {
                    'instr': 200_000,
                    'data': 5,
                    'cpu_intensity': 'low',
                    'memory_intensity': 'high',
                    'task_class': 'CPU-light, memory-intensive'
                },
                'RT4': {
                    'instr': 500_000,
                    'data': 0.5,
                    'cpu_intensity': 'low',
                    'memory_intensity': 'low',
                    'task_class': 'CPU-light, memory-light'
                }
            },
            # Small Write Tasks
            'small_write_tasks': {
                'WT3': {
                    'instr': 500_000,
                    'data': 5,
                    'cpu_intensity': 'low',
                    'io_intensity': 'high',
                    'task_class': 'CPU-light, I/O-intensive'
                },
                'WT4': {
                    'instr': 200_000,
                    'data': 0.2,
                    'cpu_intensity': 'low',
                    'io_intensity': 'low',
                    'task_class': 'CPU-light, I/O-light'
                }
            }
        }

    def generate_large_task_set(self, total_tasks: int, large_task_percentage: float = 0.7) -> Dict[str, Any]:
        """
        Generate a comprehensive set of tasks with detailed categorization
        """
        # Calculate number of each type
        num_large_tasks = int(total_tasks * large_task_percentage)
        num_small_tasks = total_tasks - num_large_tasks

        # Generate tasks
        tasks = []
        task_id = 1

        # Generate large tasks
        large_tasks = self._generate_tasks(num_large_tasks, "large", task_id)
        tasks.extend(large_tasks)
        task_id += num_large_tasks

        # Generate small tasks
        small_tasks = self._generate_tasks(num_small_tasks, "small", task_id)
        tasks.extend(small_tasks)

        # Shuffle tasks to randomize their order
        random.shuffle(tasks)

        # Categorize tasks
        categorized_tasks = self._categorize_tasks(tasks)

        # Prepare task set metadata
        task_set_metadata = {
            "total_tasks": total_tasks,
            "large_task_percentage": large_task_percentage,
            "large_tasks": num_large_tasks,
            "small_tasks": num_small_tasks,
            "task_distribution": {
                "read_tasks": sum(1 for task in tasks if task.get('type') == 'read'),
                "write_tasks": sum(1 for task in tasks if task.get('type') == 'write')
            }
        }

        return {
            "metadata": task_set_metadata,
            "categorized_tasks": categorized_tasks,
            "raw_tasks": tasks
        }

    def _generate_tasks(self, num_tasks: int, size: str, start_id: int) -> List[Dict]:
        """Generate tasks of a specific size"""
        tasks = []

        for i in range(num_tasks):
            # Randomly choose between read and write tasks (50-50 distribution)
            task_type = random.choice(["read", "write"])

            # Get appropriate config based on size and type
            config_key = f"{size}_{task_type}_tasks"
            possible_tasks = self.task_configs[config_key]

            # Randomly select a task configuration
            task_name = random.choice(list(possible_tasks.keys()))
            task_config = possible_tasks[task_name]

            # Create task dictionary
            task = {
                "id": start_id + i,
                "task_name": task_name,
                "size": size,
                "type": task_type,
                "instructions": task_config['instr'],
                "data_size": task_config['data'],
                "arrival_time": random.randint(0, 1000),  # Random arrival time
                "status": "pending",
                **{k: v for k, v in task_config.items() if k not in ['instr', 'data']}
            }
            tasks.append(task)

        return tasks

    def _categorize_tasks(self, tasks: List[Dict]) -> Dict[str, List[Dict]]:
        """
        Categorize tasks by their specific characteristics
        """
        categorized = defaultdict(list)

        # Categorize by task names (RT1, RT2, etc.)
        for task in tasks:
            categorized[task['task_name']].append(task)

        return dict(categorized)

def generate_and_save_task_set(total_tasks: int = 1500, large_task_percentage: float = 0.7):
    """
    Generate task set, save to file, and print detailed categorization
    """
    # Create task generator
    generator = TaskGenerator()

    # Generate tasks
    task_set = generator.generate_large_task_set(
        total_tasks=total_tasks,
        large_task_percentage=large_task_percentage
    )

    # Print detailed categorization
    print("\n--- DETAILED TASK CATEGORIZATION ---")
    for task_category, tasks in task_set['categorized_tasks'].items():
        print(f"\n{task_category} Tasks:")
        print(f"Total {task_category} Tasks: {len(tasks)}")
        print("Sample Task Details:")
        for task in tasks[:3]:  # Print first 3 tasks of each category
            print("\nTask Details:")
            for key, value in task.items():
                print(f"{key}: {value}")
        print("-" * 50)

    # Print overall metadata
    print("\n--- TASK SET METADATA ---")
    print(json.dumps(task_set['metadata'], indent=2))

    # Save to JSON
    save_path = 'fcfs_task_set.json'
    with open(save_path, 'w') as f:
        json.dump(task_set, f, indent=2)

    print(f"\nFull task set saved to: {save_path}")

    return task_set

# Run the task generation
if __name__ == "__main__":
    generate_and_save_task_set()



--- DETAILED TASK CATEGORIZATION ---

WT1 Tasks:
Total WT1 Tasks: 270
Sample Task Details:

Task Details:
id: 156
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 401
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive

Task Details:
id: 951
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 637
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive

Task Details:
id: 123
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 52
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive
--------------------------------------------------

RT2 Tasks:
Total RT2 Tasks: 253
Sample Task Details:

Task Details:
id: 462
task_name: RT2
size: large
type: read
instructions: 4000000
data_size: 0.2
arrival_time: 656
status: pending
cpu_intensity: high
memory_intensity: l

Once we have our stopping criterion, we can finally run our simulation by creating an instance of the `Simulator` class, loading a dataset, and calling the `run_model()` method.

In [7]:

# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

# Executing the simulation
simulator.run_model()

# Checking the placement output
for service in Service.all():
    print(f"{service}. Host: {service.server}")

Service_1. Host: EdgeServer_1
Service_2. Host: EdgeServer_1
Service_3. Host: EdgeServer_1
Service_4. Host: EdgeServer_1
Service_5. Host: EdgeServer_1
Service_6. Host: EdgeServer_1
